# Data exploration

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import beautifulsoup4 as bs

## Data cleansing

In [2]:
data = pd.read_csv("datasets\combined_csv.csv", low_memory=False, dtype="object")
data.head(1)

,Id,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount
0,68186938,68186998.0,2021-06-29 23:59:26,-2,31,"<p>I have a function that takes a string, brea...",5257353.0,"Comparing string arrays for matching elements,...",<python>,2,1


In [3]:
print((data.isnull().mean() *  100).round(2))

Id                   0.00
AcceptedAnswerId    47.76
CreationDate         0.00
Score                0.00
ViewCount            0.00
Body                 0.00
OwnerUserId          1.29
Title                0.00
Tags                 0.00
AnswerCount          0.00
CommentCount         0.00
dtype: float64


In [4]:
data["AcceptedAnswerId"].fillna(value="NoAnswer", inplace=True)
print(data.isnull().mean()*100).round(2)

Id                  0.000000
AcceptedAnswerId    0.000000
CreationDate        0.000000
Score               0.000000
ViewCount           0.000000
Body                0.000000
OwnerUserId         1.288832
Title               0.000000
Tags                0.000000
AnswerCount         0.000000
CommentCount        0.000000
dtype: float64


AttributeError: 'NoneType' object has no attribute 'round'